<a href="https://colab.research.google.com/github/Uweeseg1/DataAnalyticsProject/blob/master/PARTII_EDA_and_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install numpy_financial

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd 
import os
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from google.colab import drive
import numpy_financial as npf


## Loading and Reading the data

In [ ]:
# import numpy_financial as npf

In [ ]:
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
! ls

loan-data-apr-2018-snap   Q1_2018.csv  Q2_2019.csv  Q4_2018.csv
loan-data-jul-2019-snap   Q1_2019.csv  Q3_2018.csv  Q4_2019.csv
PARTII_EDA_and_ETL.ipynb  Q2_2018.csv  Q3_2019.csv


In [ ]:
# upload DATA

Q1_19 = pd.read_csv("Q1_2019.csv",low_memory=False)
Q2_19 = pd.read_csv("Q2_2019.csv",low_memory=False)
Q3_19 = pd.read_csv("Q3_2019.csv",low_memory=False)
Q4_19 = pd.read_csv("Q4_2019.csv",low_memory=False)

# upload DATA

Q1_18 = pd.read_csv("Q1_2018.csv",low_memory=False)
Q2_18 = pd.read_csv("Q2_2018.csv",low_memory=False)
Q3_18 = pd.read_csv("Q3_2018.csv",low_memory=False)
Q4_18 = pd.read_csv("Q4_2018.csv",low_memory=False)

In [ ]:
print(Q1_19.shape,Q1_18.shape)
print(Q2_19.shape,Q2_18.shape)
print(Q3_19.shape,Q3_18.shape)
print(Q4_19.shape,Q4_18.shape)

(133887, 151) (133887, 152)
(97854, 151) (97854, 152)
(99120, 151) (99120, 152)
(103546, 151) (103546, 152)


# Concatenating dataframes into annual categories

In [ ]:
# making 2 big Date Frames
all_year_19 = pd.concat([Q1_19,Q2_19,Q3_19,Q4_19])
all_year_18 = pd.concat([Q1_18,Q2_18,Q3_18,Q4_18])

In [ ]:
all_year_19.payment_lifetime.describe()


count    434407.000000
mean        658.441839
std        1529.652635
min      -42705.000000
25%         425.000000
50%         762.000000
75%        1005.000000
max        1247.000000
Name: payment_lifetime, dtype: float64

## Bellow we will begin b y filtering out columns with 90% or nore N/A values


In [ ]:
na_percentages = all_year_19.isna().mean()

In [ ]:
na_percentages

id                       0.000000
member_id                1.000000
loan_amnt                0.000000
funded_amnt              0.000000
funded_amnt_inv          0.000000
                           ...   
settlement_percentage    0.971055
settlement_term          0.971055
payment_lifetime         0.000000
months                   0.000000
monthly_paymnent         0.000000
Length: 153, dtype: float64

In [ ]:
cols_to_drop = na_percentages[na_percentages >= 0.9].index
all_year_19_wihtout_90nas = all_year_19.drop(cols_to_drop, axis=1)

In [ ]:
cols_to_check = na_percentages[(na_percentages >= 0.05) & (na_percentages <= 0.9) ].index


In [ ]:
len(cols_to_check)

11

In [ ]:
all_year_19_columns_to_check = all_year_19_wihtout_90nas[cols_to_check]# axis=1]

In [ ]:
all_year_19_columns_to_check.columns

Index(['emp_title', 'emp_length', 'title', 'mths_since_last_delinq',
       'mths_since_last_record', 'next_pymnt_d', 'mths_since_last_major_derog',
       'il_util', 'mths_since_recent_bc_dlq', 'mths_since_recent_inq',
       'mths_since_recent_revol_delinq'],
      dtype='object')

In [ ]:
len(all_year_19_columns_to_check.columns)

147

In [ ]:


# for low values appiring by missing date (start or finish) 
# i assing max time (36 months ) for all negative values:
## it will only hurt (lower) relised return as 
## we asuming those payments are recived as late as possible
### values with 0 get 1 (its best to check in 18 tabbles)
months_vector = list(round(all_year_19.payment_lifetime/30.4))
for i in range(len(months_vector)) : 
    if months_vector[i] < 0 :
        months_vector[i] = 36
    # ! problem with 0 values ! change this ! 
    elif months_vector[i] == 0 : 
        months_vector[i] = 1
    # ! problem with 0 values ! change this ! 

In [ ]:
# now lets take another look
sum(np.array(months_vector) <= 0 )
all_year_19["months"] = months_vector
all_year_19.months.describe()

count    434407.000000
mean         23.356633
std          11.247674
min           1.000000
25%          14.000000
50%          26.000000
75%          33.000000
max          41.000000
Name: months, dtype: float64

In [ ]:
# lets calculate the monthly payments 
all_year_19["monthly_paymnent"] = all_year_19["total_pymnt"] / all_year_19["months"]

In [ ]:
all_year_19.loan_status.unique()

array(['Fully Paid', 'Current', 'Charged Off', 'Late (31-120 days)',
       'Late (16-30 days)', 'In Grace Period', 'Default'], dtype=object)

In [ ]:
# now we can make shpithzer Monthly payments 
shpizer_payments = []

for _, row in all_year_19.iterrows():
    pymnts = [row['funded_amnt']*-1]  
    pymnts += [row['monthly_paymnent']] * int(row['months'])  
    shpizer_payments.append(pymnts) 

In [ ]:
# Monthly rate !
realised_rate = []
for i in shpizer_payments : 
    realised_rate.append(npf.irr(i))
# let's turn it to yearly
yearly_realised_rates = ((np.array(realised_rate) + 1)**12) -1 